# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 2 2022 9:35AM,245601,10,PRF-36239,Bio-PRF,Released
1,Sep 2 2022 9:35AM,245601,10,PRF-36290,Bio-PRF,Released
2,Sep 2 2022 9:35AM,245601,10,PRF-36306,Bio-PRF,Released
3,Sep 2 2022 9:35AM,245601,10,PRF-36311,Bio-PRF,Released
4,Sep 2 2022 9:35AM,245601,10,PRF-36315,Bio-PRF,Released
5,Sep 2 2022 9:35AM,245601,10,PRF-36322,Bio-PRF,Released
6,Sep 2 2022 9:35AM,245601,10,PRF-36329,Bio-PRF,Released
7,Sep 2 2022 9:35AM,245601,10,PRF-36332,Bio-PRF,Released
8,Sep 2 2022 9:35AM,245601,10,PRF-36335,Bio-PRF,Released
9,Sep 2 2022 9:35AM,245601,10,PRF-36338,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,245602,Executing,1
44,245602,Released,9
45,245603,Released,1
46,245604,Released,1
47,245605,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245601,NaN,NaN,14.0
245602,NaN,1.0,9.0
245603,NaN,NaN,1.0
245604,NaN,NaN,1.0
245605,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,6.0,1.0,3.0
245455,3.0,2.0,1.0
245478,26.0,0.0,15.0
245525,0.0,0.0,1.0
245545,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245453,6,1,3
245455,3,2,1
245478,26,0,15
245525,0,0,1
245545,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,6,1,3
1,245455,3,2,1
2,245478,26,0,15
3,245525,0,0,1
4,245545,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245453,6,1,3
1,245455,3,2,1
2,245478,26,,15
3,245525,,,1
4,245545,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 2 2022 9:35AM,245601,10,Bio-PRF
14,Sep 2 2022 9:34AM,245605,10,Eywa Pharma Inc.
24,Sep 2 2022 9:29AM,245604,10,"Citieffe, Inc."
25,Sep 2 2022 9:27AM,245603,10,"Nextsource Biotechnology, LLC"
26,Sep 2 2022 9:25AM,245602,12,Hush Hush
36,Sep 2 2022 9:17AM,245586,20,Alumier Labs Inc.
41,Sep 2 2022 9:03AM,245600,10,Eywa Pharma Inc.
43,Sep 2 2022 8:52AM,245598,10,ISDIN Corporation
98,Sep 2 2022 8:52AM,245597,10,ISDIN Corporation
160,Sep 2 2022 8:49AM,245599,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 2 2022 9:35AM,245601,10,Bio-PRF,,,14
1,Sep 2 2022 9:34AM,245605,10,Eywa Pharma Inc.,,,10
2,Sep 2 2022 9:29AM,245604,10,"Citieffe, Inc.",,,1
3,Sep 2 2022 9:27AM,245603,10,"Nextsource Biotechnology, LLC",,,1
4,Sep 2 2022 9:25AM,245602,12,Hush Hush,,1,9
5,Sep 2 2022 9:17AM,245586,20,Alumier Labs Inc.,,,5
6,Sep 2 2022 9:03AM,245600,10,Eywa Pharma Inc.,,,2
7,Sep 2 2022 8:52AM,245598,10,ISDIN Corporation,,,55
8,Sep 2 2022 8:52AM,245597,10,ISDIN Corporation,,,62
9,Sep 2 2022 8:49AM,245599,10,Eywa Pharma Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 9:35AM,245601,10,Bio-PRF,14,,
1,Sep 2 2022 9:34AM,245605,10,Eywa Pharma Inc.,10,,
2,Sep 2 2022 9:29AM,245604,10,"Citieffe, Inc.",1,,
3,Sep 2 2022 9:27AM,245603,10,"Nextsource Biotechnology, LLC",1,,
4,Sep 2 2022 9:25AM,245602,12,Hush Hush,9,1,
5,Sep 2 2022 9:17AM,245586,20,Alumier Labs Inc.,5,,
6,Sep 2 2022 9:03AM,245600,10,Eywa Pharma Inc.,2,,
7,Sep 2 2022 8:52AM,245598,10,ISDIN Corporation,55,,
8,Sep 2 2022 8:52AM,245597,10,ISDIN Corporation,62,,
9,Sep 2 2022 8:49AM,245599,10,Eywa Pharma Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 9:35AM,245601,10,Bio-PRF,14,,
1,Sep 2 2022 9:34AM,245605,10,Eywa Pharma Inc.,10,,
2,Sep 2 2022 9:29AM,245604,10,"Citieffe, Inc.",1,,
3,Sep 2 2022 9:27AM,245603,10,"Nextsource Biotechnology, LLC",1,,
4,Sep 2 2022 9:25AM,245602,12,Hush Hush,9,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 9:35AM,245601,10,Bio-PRF,14.0,NaN,NaN
1,Sep 2 2022 9:34AM,245605,10,Eywa Pharma Inc.,10.0,NaN,NaN
2,Sep 2 2022 9:29AM,245604,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Sep 2 2022 9:27AM,245603,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Sep 2 2022 9:25AM,245602,12,Hush Hush,9.0,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 2 2022 9:35AM,245601,10,Bio-PRF,14.0,0.0,0.0
1,Sep 2 2022 9:34AM,245605,10,Eywa Pharma Inc.,10.0,0.0,0.0
2,Sep 2 2022 9:29AM,245604,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Sep 2 2022 9:27AM,245603,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Sep 2 2022 9:25AM,245602,12,Hush Hush,9.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4666127,331.0,1.0,1.0
12,736763,14.0,1.0,0.0
15,491072,30.0,0.0,26.0
16,982244,3.0,1.0,0.0
17,491165,2.0,0.0,0.0
20,2209915,17.0,11.0,12.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4666127,331.0,1.0,1.0
1,12,736763,14.0,1.0,0.0
2,15,491072,30.0,0.0,26.0
3,16,982244,3.0,1.0,0.0
4,17,491165,2.0,0.0,0.0
5,20,2209915,17.0,11.0,12.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,331.0,1.0,1.0
1,12,14.0,1.0,0.0
2,15,30.0,0.0,26.0
3,16,3.0,1.0,0.0
4,17,2.0,0.0,0.0
5,20,17.0,11.0,12.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,331.0
1,12,Released,14.0
2,15,Released,30.0
3,16,Released,3.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,20
Status,,,,,,
Completed,1.0,0.0,26.0,0.0,0.0,12.0
Executing,1.0,1.0,0.0,1.0,0.0,11.0
Released,331.0,14.0,30.0,3.0,2.0,17.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,20
0,Completed,1.0,0.0,26.0,0.0,0.0,12.0
1,Executing,1.0,1.0,0.0,1.0,0.0,11.0
2,Released,331.0,14.0,30.0,3.0,2.0,17.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,20
0,Completed,1.0,0.0,26.0,0.0,0.0,12.0
1,Executing,1.0,1.0,0.0,1.0,0.0,11.0
2,Released,331.0,14.0,30.0,3.0,2.0,17.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()